In [1]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, Input, BatchNormalization, concatenate
from keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2
import tensorflow as tf
import imgaug as ia
import imgaug.augmenters as iaa
from albumentations import (
    RandomRotate90,Flip
)
import scipy.io
import difflib
from scipy import spatial
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import spatial

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print('This should list a GPU: ',tf.test.gpu_device_name())
os.chdir('/home/xx652230/Downloads/')

This should list a GPU:  /device:GPU:0


In [2]:
t=pd.read_excel("./amigos/personalities.xlsx")
traits=[]
for i in range(1,31):
    if i!=8 and i!=28:
        #if t[i][0]>5:
            trait=[t[i][j] for j in range(5)]
            traits.append(trait)
        #else:
         #   traits.append(0)

In [3]:
index={}
for i in range(0,31):
    if i>7 and i<12:
        index[i]=i-2
    elif i> 12 and i<28:
        index[i]=i-2
    elif i>28:
        index[i]=i-3
    else:
        index[i]=i-1

In [4]:
num_features=64
modele = Sequential([
    Conv2D(filters=64, kernel_size=5, activation='relu', input_shape=(120, 120, 1)),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=16, kernel_size=5, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])
modele.load_weights("/home/xx652230/deap.hdf5")
modele.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])

In [5]:
num_features = 64
width, height = 48, 48
modelf = Sequential([
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 2
        Dropout(0.5),

        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 6
        BatchNormalization(),
        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 7
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 8
        Dropout(0.5),

        Flatten(),

        Dense(2 * 2 * 2 * num_features, activation='relu'),  # 9
        Dropout(0.4),
        Dense(2 * 2 * num_features, activation='relu'),  # 10
        Dropout(0.4),
        Dense(2 * num_features, activation='relu'),  # 10
        Dropout(0.5),

        Dense(7, activation='softmax')

])
modelf.load_weights("./model/face/final_model_face.hdf5") 
modelf.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])


In [ ]:
a1=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b1=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined = concatenate([a1.output, b1.output])
z1 = Dense(256, activation="relu",)(combined)
z1 = Dense(1,activation="sigmoid")(z1)
model2 = Model(inputs=[a1.input, b1.input], outputs=z1)
for i in range(len(model2.layers)-3):
    model2.layers[i].trainable=False
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model2.load_weights("./loss/creative.hdf5") 


In [ ]:
from keras.layers import TimeDistributed,Reshape
a=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
video=Input(shape=(None,120,120,1))
video1=Input(shape=(None,48,48,1))
#b=Model(inputs=model_dude.inputs,outputs=model_dude.layers[-8].output)
mod=Sequential()
mod.add(video)
mod.add(TimeDistributed(a))
#mod.add(TimeDistributed(Flatten()))
mod.add(LSTM(128,return_sequences=True))
mod.add(TimeDistributed(Dense(128, activation="relu",name="eeg")))

b=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)

#b=Model(inputs=model_dude.inputs,outputs=model_dude.layers[-8].output)
mod1=Sequential()
mod1.add(video1)
mod1.add(TimeDistributed(b))
#mod1.add(TimeDistributed(Flatten()))
mod1.add(LSTM(128,return_sequences=True))
mod1.add(TimeDistributed(Dense(128, activation="relu"),name="face"))
cont=concatenate([mod.output,mod1.output])
z=TimeDistributed(Dense(128, activation="relu"))(cont)
z =TimeDistributed(Dense(1,activation="sigmoid"))(z)
model_dope=Model(inputs=[mod.input,mod1.input],outputs=z)
for i in range(len(model_dope.layers)-8):
    model_dope.layers[i].trainable=False
#model_dope.load_weights("./results/cnnlstmconcat_p30.hdf5")
model_dope.compile(optimizer="adam", loss='binary_crossentropy' , metrics=['accuracy'])
#w1=[i*0.4 for i in model_dope.layers[6].get_weights()] #eeg
#model_dope.layers[6].set_weights(w1)
#w2=[i*0.6 for i in model_dope.layers[7].get_weights()] #face
#model_dope.layers[7].set_weights(w2)


In [6]:
def read_csv(path,d1,d2):
    file_read=pd.read_csv(path)
    construct=[]
    #construct.append([i for i in file_read.columns])
    for i in range(len(file_read)):
        construct.append(file_read.loc[i,:])
    if d2!=0:    
        file1=np.reshape(np.array(construct)[39:],(len(construct)-39,d1,d2,1))  
        return file1
    else:
        file1=np.reshape(np.array(construct)[39:],(len(construct)-39,d1))
        return file1

In [7]:

def round_off(sample):
    trait_final=0.0
    #for i in range(len(sample)):
    if sample[0]>5:
        trait_final=1.0
    else:
        trait_final=0.0
    return trait_final 
def round_off1(sample):
    trait_final=[0.0]*5
    t=np.where(sample==max(sample))
    for i in t[0]:
        trait_final[i]=1.0
    return trait_final

def trait_output(see_file,ind):
    y_person=[]
    length=len(see_file)
    for i in range(length):
        y_person.append(round_off(traits[ind]))
        
    return y_person

In [42]:
imps=["eeg","face"]
eeg=[]
face=[]
eeg1=[]
face1=[]
for j in range(1,21):
    notin=[8,12,28]
    if j not in notin :
      if j<10: 
        for i in imps:
            if i=="eeg":
                eeg.append(read_csv("./SortedSamples/participants/"+str(i)+"P0"+str(j)+".csv",120,120))  
                eeg1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",120,120))     

            elif i=="face":   
                face.append(read_csv("./SortedSamples/participants/"+str(i)+"P0"+str(j)+".csv",48,48))
                face1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",48,48))

           
      elif  j>9: 
        for i in imps:
            if i=="eeg":
                eeg.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",120,120))
                eeg1.append(read_csv("./t/"+str(i)+"P"+str(j)+".csv",120,120))     

              
            elif i=="face":   
                face.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",48,48))
                face1.append(read_csv("./t/"+str(i)+"P"+str(j)+".csv",48,48))

            elif i=="quad":
                quad_out.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",5,0))
     


In [41]:
imps=["eeg","face","quad","model"]
y_traits=[]
y_traits1=[]
for j in range(1,21):
    notin=[8,12,28]
    if j not in notin :
      if j<10: 
        for i in imps:
            if i=="eeg":
                if j>8:
                    y_traits.append(trait_output(eeg[j-2],index[j]))
                    y_traits1.append(trait_output(eeg1[j-2],index[j]))

                else:   
                    y_traits.append(trait_output(eeg[j-1],index[j]))
                    y_traits1.append(trait_output(eeg1[j-1],index[j]))


      elif  j>9: 
        for i in imps:
            if i=="eeg":
                if j>11 and j<29:
                    y_traits.append(trait_output(eeg[j-3],index[j]))
                    y_traits1.append(trait_output(eeg1[j-3],index[j]))

                elif j>27:   
                    y_traits.append(trait_output(eeg[j-4],index[j]))
                    y_traits1.append(trait_output(eeg1[j-4],index[j]))

                elif j<12:   
                    y_traits.append(trait_output(eeg[j-2],index[j]))    
                    y_traits1.append(trait_output(eeg1[j-2],index[j]))


IndexError: tuple index out of range

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=None, shuffle=True)
acc_per_fold=[]
loss_per_fold=[]
for train_index,test_index in kf.split(np.array(eeg_copy)):
    x1=np.concatenate([eeg_copy[i] for i in train_index]) 
    x2=np.concatenate([face_copy[i] for i in train_index])
    y1=np.concatenate([y_traits[i] for i in train_index])
    x4=np.concatenate([eeg_copy[i] for i in test_index])
    x5=np.concatenate([face_copy[i] for i in test_index])
    y6=np.concatenate([y_traits[i] for i in test_index])
    history = model2.fit([np.array(x1),np.array(x2)], np.array(y1), validation_data=([np.array(x4),np.array(x5)], np.array(y6)),epochs=15, verbose=1,batch_size=256)  
    scores = model2.evaluate([np.array(x4),np.array(x5)], np.array(y6), verbose=0)
    print(f'Score for fold : {model2.metrics_names[0]} of {scores[0]}; {model2.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    

In [ ]:
def _load_data(data, steps = 40):  
    docX, docY = [], []
    for i in range(0, int(data.shape[0]/steps-1)):
        docX.append(data[i*steps:(i+1)*steps,:])
        #docY.append(data[(i*steps+1):((i+1)*steps+1),:])
    return np.array(docX)

In [ ]:
def _load_data1(data, steps = 40):  
    docX, docY = [], []
    for i in range(0, int(data.shape[0]/steps-1)):
        docX.append(data[i*steps:(i+1)*steps])
        #docY.append(data[(i*steps+1):((i+1)*steps+1),:])
    return np.array(docX)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=None, shuffle=True)
acc_per_fold=[]
loss_per_fold=[]
matrix=[]
for train_index,test_index in kf.split(np.array(eeg)):
    print(train_index,test_index)
    emotion_list=["Q1","Q2","Q3","Q4","Neutral"]
    x1=np.concatenate([eeg[i] for i in train_index])  
    x2=np.concatenate([face[i] for i in train_index])
    y1=np.concatenate([y_traits[i] for i in train_index])
    x4=np.concatenate([eeg1[i] for i in test_index])
    x5=np.concatenate([face1[i] for i in test_index])
    y6=np.concatenate([y_traits1[i] for i in test_index])
    xx1=_load_data(np.array(x1))
    xx2=_load_data(np.array(x2))
    yy1=_load_data1(np.array(y1))
    xx4=_load_data(np.array(x4))
    xx5=_load_data(np.array(x5))
    yy6=_load_data1(np.array(y6))
    history = model_dope.fit([np.array(xx1),np.array(xx2)], np.array(yy1), validation_data=([np.array(xx4),np.array(xx5)], np.array(yy6)),epochs=5, verbose=1,steps_per_epoch=10)  
    scores = model_dope.evaluate([np.array(xx4),np.array(xx5)], np.array(yy6), verbose=0)          
    print(f'Score for fold : {model_dope.metrics_names[0]} of {scores[0]}; {model_dope.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

# Task 2: Emotion personality relationships

In [10]:
a1=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b1=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined1 = concatenate([a1.output, b1.output])
z1 = Dense(256, activation="relu")(combined1)
z1 = Dense(1, activation="sigmoid")(z1)
model1 = Model(inputs=[a1.input, b1.input], outputs=z1)
for i in range(len(model1.layers)-3):
    model1.layers[i].trainable=False
    
model1.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
model1.load_weights("./loss/extroversion.hdf5") 

In [11]:
a2=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b2=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined2 = concatenate([a2.output, b2.output])
z2 = Dense(256, activation="relu")(combined2)
z2 = Dense(1, activation="sigmoid")(z2)
model2 = Model(inputs=[a2.input, b2.input], outputs=z2)
for i in range(len(model2.layers)-3):
    model2.layers[i].trainable=False
    
model2.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
model2.load_weights("./loss/agreeable.hdf5") 

In [12]:
a3=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b3=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined3 = concatenate([a3.output, b3.output])
z3 = Dense(256, activation="relu")(combined3)
z3 = Dense(1, activation="sigmoid")(z3)
model3 = Model(inputs=[a3.input, b3.input], outputs=z3)
for i in range(len(model3.layers)-3):
    model3.layers[i].trainable=False
    
model3.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
model3.load_weights("./loss/conscientious.hdf5") 

In [13]:
a4=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b4=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined4 = concatenate([a4.output, b4.output])
z4 = Dense(256, activation="relu")(combined4)
z4 = Dense(1, activation="sigmoid")(z4)
model4 = Model(inputs=[a4.input, b4.input], outputs=z4)
for i in range(len(model4.layers)-3):
    model4.layers[i].trainable=False
    
model4.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
model4.load_weights("./loss/emotional.hdf5")

In [14]:
a5=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b5=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined5 = concatenate([a5.output, b5.output])
z5 = Dense(256, activation="relu")(combined5)
z5 = Dense(1, activation="sigmoid")(z5)
model5 = Model(inputs=[a5.input, b5.input], outputs=z5)
for i in range(len(model5.layers)-3):
    model5.layers[i].trainable=False
    
model5.compile(loss="categorical_crossentropy", optimizer='adam',metrics=['accuracy'])
model5.load_weights("./loss/creative.hdf5")

In [43]:
x1all=np.concatenate([eeg1[i] for i in range(len(eeg1))])
x2all=np.concatenate([face1[i] for i in range(len(eeg1))])
#y1=np.concatenate([y_traits[i] for i in  l3])
#x4=np.concatenate([eeg1[i] for i in  l2])
#x5=np.concatenate([face1[i] for i in  l2])
#y6=np.concatenate([y_traits[i] for i in  l2])

In [44]:
from numpy import corrcoef

quad1=[]
quad2=[]
quad3=[]
quad4=[]
neutral=[]
quad1e=[]
quad2e=[]
quad3e=[]
quad4e=[]
neutrale=[]
personal1=model1.predict([np.array(x1all),np.array(x2all)])
personal2=model2.predict([np.array(x1all),np.array(x2all)])
personal3=model3.predict([np.array(x1all),np.array(x2all)])
personal4=model4.predict([np.array(x1all),np.array(x2all)])
personal5=model5.predict([np.array(x1all),np.array(x2all)])
alls=[[personal1[i][0],personal2[i][0],personal3[i][0],personal4[i][0],personal5[i][0]] for i in range(len(personal1))]
personal=modele.predict(np.array(x1all))
for i in range(len(personal)):
    index=np.where(personal[i]==max(personal[i]))
    if index[0][0]==0:
        quad1.append([j for j in alls[i]])
        quad1e.append([j for j in personal[i]])
        #quad1e.append([1,0,0,0,0])
    elif index[0][0]==1:
        quad2.append([j for j in alls[i]])
        quad2e.append([j for j in personal[i]])
        #quad2e.append([0,1,0,0,0])
        
    elif index[0][0]==2:
        quad3.append([j for j in alls[i]])
        quad3e.append([j for j in personal[i]])
        #quad3e.append([0,0,1,0,0])

    elif index[0][0]==3:
        quad4.append([j for j in alls[i]])
        quad4e.append([j for j in personal[i]])
        #quad4e.append([0,0,0,1,0])

    else:
        neutral.append([j for j in alls[i]])
        neutrale.append([j for j in personal[i]])
        #neutrale.append([0,0,0,0,1])



In [66]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr

k=[]
d=[]
for i in range(len(neutral)):
    p1,p2=pearsonr(neutral[i],neutrale[i])
    if p2<0.05:
        k.append(neutral[i])
        d.append([p1,p2])

In [67]:
dic={}
for it in range(5):
    dic[it]=0
for j in k:
    t=np.where(j==max(j))[0][0]
    dic[t]=dic[t]+1

In [68]:
dic

{0: 0, 1: 19, 2: 12, 3: 2, 4: 2}